In [1]:
!pip install pyxlsb

In [2]:
# 📦 Import libraries
import pandas as pd
from difflib import SequenceMatcher
import re
from itertools import combinations
from tqdm import tqdm

# ✅ Load your XLSB file
# Update this path based on how Kaggle mounts your uploaded file
file_path = "/kaggle/input/invoice-dump-24-25-xlsb/Invoice_Dump_24-25.xlsb"
df = pd.read_excel(file_path, sheet_name=0, engine='pyxlsb')

# ✅ Clean and normalize necessary columns
df = df[['PAYEE_NAME', 'INVOICE_NO', 'TOTAL_AMOUNT']].dropna().copy()
df['INVOICE_NO'] = df['INVOICE_NO'].astype(str)
df['PAYEE_NAME'] = df['PAYEE_NAME'].astype(str)

def normalize(text):
    return re.sub(r'[^A-Za-z0-9]', '', text).upper()

df['Normalized_Invoice'] = df['INVOICE_NO'].apply(normalize)
df['Normalized_Payer'] = df['PAYEE_NAME'].apply(normalize)

# ✅ Fuzzy Matching (Grouped to Reduce Load)
matches = []
grouped = df.groupby(['Normalized_Payer', 'TOTAL_AMOUNT'])

for (payer, amount), group_df in tqdm(grouped, desc="Processing Groups"):
    inv_list = group_df[['PAYEE_NAME', 'INVOICE_NO', 'Normalized_Invoice']].drop_duplicates().values.tolist()
    for (p1, i1, n1), (p2, i2, n2) in combinations(inv_list, 2):
        if n1 != n2:
            sim = SequenceMatcher(None, n1, n2).ratio()
            if sim >= 0.9:
                matches.append((p1, i1, i2, amount, round(sim, 2)))

# ✅ Create DataFrames
fuzzy_df = pd.DataFrame(matches, columns=['Payer Name', 'Invoice_1', 'Invoice_2', 'Amount', 'Similarity'])
matched_invoices = set(fuzzy_df['Invoice_1']).union(set(fuzzy_df['Invoice_2']))
matched_details = df[df['INVOICE_NO'].isin(matched_invoices)]

# ✅ Save to Excel (Kaggle working directory)
output_path = "/kaggle/working/Invoice_Fuzzy_Report_FULL.xlsx"
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df[['PAYEE_NAME', 'INVOICE_NO', 'TOTAL_AMOUNT']].to_excel(writer, sheet_name="Raw_Data", index=False)
    fuzzy_df.to_excel(writer, sheet_name="Fuzzy_Matches", index=False)
    matched_details.to_excel(writer, sheet_name="Matched_Details", index=False)

print(f"✅ Done! Download from: {output_path}")


Processing Groups: 100%|██████████| 90116/90116 [04:32<00:00, 330.71it/s]


✅ Done! Download from: /kaggle/working/Invoice_Fuzzy_Report_FULL.xlsx


In [3]:
# 📦 Required Libraries
import pandas as pd
from difflib import SequenceMatcher
import re
from itertools import combinations
from tqdm import tqdm

# ✅ Load Excel XLSB File (replace with your actual file path from sidebar)
file_path = "/kaggle/input/invoice-dump-24-25-xlsb/Invoice_Dump_24-25.xlsb"
df = pd.read_excel(file_path, sheet_name=0, engine='pyxlsb')

# ✅ Keep only required columns and drop missing
df = df[['PAYEE_CODE', 'INVOICE_NO', 'TOTAL_AMOUNT']].dropna().copy()

# Convert types
df['INVOICE_NO'] = df['INVOICE_NO'].astype(str)
df['PAYEE_CODE'] = df['PAYEE_CODE'].astype(str)

# ✅ Normalize text for fuzzy comparison
def normalize(text):
    return re.sub(r'[^A-Za-z0-9]', '', text).upper()

df['Normalized_Invoice'] = df['INVOICE_NO'].apply(normalize)
df['Normalized_Payer'] = df['PAYEE_CODE'].apply(normalize)

# ✅ Fuzzy Matching - Optimized by Grouping Payer + Amount
matches = []
grouped = df.groupby(['Normalized_Payer', 'TOTAL_AMOUNT'])

for (payer, amount), group_df in tqdm(grouped, desc="🔍 Matching Groups"):
    inv_list = group_df[['PAYEE_CODE', 'INVOICE_NO', 'Normalized_Invoice']].drop_duplicates().values.tolist()
    for (payer1, inv1, norm1), (payer2, inv2, norm2) in combinations(inv_list, 2):
        if norm1 != norm2:
            sim = SequenceMatcher(None, norm1, norm2).ratio()
            if sim >= 0.90:
                matches.append((payer1, inv1, inv2, amount, round(sim, 2)))

# ✅ Convert Matches to DataFrame
fuzzy_df = pd.DataFrame(matches, columns=['Payee_Code', 'Invoice_1', 'Invoice_2', 'Amount', 'Similarity'])

# ✅ Filter original details for all matched invoices
matched_invoices = set(fuzzy_df['Invoice_1']).union(set(fuzzy_df['Invoice_2']))
matched_details = df[df['INVOICE_NO'].isin(matched_invoices)]

# ✅ Save results to Excel (Kaggle Working Directory)
output_path = "/kaggle/working/Invoice_Fuzzy_Report_PayerCode.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df[['PAYEE_CODE', 'INVOICE_NO', 'TOTAL_AMOUNT']].to_excel(writer, sheet_name="Raw_Data", index=False)
    fuzzy_df.to_excel(writer, sheet_name="Fuzzy_Matches", index=False)
    matched_details.to_excel(writer, sheet_name="Matched_Details", index=False)

print(f"✅ Done! Download the file from: {output_path}")


🔍 Matching Groups: 100%|██████████| 88321/88321 [04:29<00:00, 327.46it/s]


✅ Done! Download the file from: /kaggle/working/Invoice_Fuzzy_Report_PayerCode.xlsx


In [4]:
# 📦 Required Libraries
import pandas as pd
from difflib import SequenceMatcher
import re
from itertools import combinations
from tqdm import tqdm

# ✅ Load the full XLSB file (make sure the path matches your uploaded file)
file_path = "/kaggle/input/invoice-dump-24-25-xlsb/Invoice_Dump_24-25.xlsb"
raw_df = pd.read_excel(file_path, sheet_name=0, engine='pyxlsb')

# ✅ Make a working copy for fuzzy logic processing
df = raw_df[['PAYEE_CODE', 'INVOICE_NO', 'TOTAL_AMOUNT']].dropna().copy()

# Convert relevant fields to string
df['INVOICE_NO'] = df['INVOICE_NO'].astype(str)
df['PAYEE_CODE'] = df['PAYEE_CODE'].astype(str)

# ✅ Normalize text for comparison
def normalize(text):
    return re.sub(r'[^A-Za-z0-9]', '', text).upper()

df['Normalized_Invoice'] = df['INVOICE_NO'].apply(normalize)
df['Normalized_Payer'] = df['PAYEE_CODE'].apply(normalize)

# ✅ Fuzzy Matching - Group by PAYEE_CODE and Amount
matches = []
grouped = df.groupby(['Normalized_Payer', 'TOTAL_AMOUNT'])

for (payer, amount), group_df in tqdm(grouped, desc="🔍 Matching Groups"):
    inv_list = group_df[['PAYEE_CODE', 'INVOICE_NO', 'Normalized_Invoice']].drop_duplicates().values.tolist()
    for (payer1, inv1, norm1), (payer2, inv2, norm2) in combinations(inv_list, 2):
        if norm1 != norm2:
            sim = SequenceMatcher(None, norm1, norm2).ratio()
            if sim >= 0.90:
                matches.append((payer1, inv1, inv2, amount, round(sim, 2)))

# ✅ Build DataFrames for output
fuzzy_df = pd.DataFrame(matches, columns=['Payee_Code', 'Invoice_1', 'Invoice_2', 'Amount', 'Similarity'])

# ✅ Get matched details from the full raw data
matched_invoices = set(fuzzy_df['Invoice_1']).union(set(fuzzy_df['Invoice_2']))
matched_details = raw_df[raw_df['INVOICE_NO'].astype(str).isin(matched_invoices)]

# ✅ Save to Excel — All Sheets
output_path = "/kaggle/working/Invoice_Fuzzy_Report_With_Raw.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    raw_df.to_excel(writer, sheet_name="Raw_Data", index=False)              # Full original raw data
    fuzzy_df.to_excel(writer, sheet_name="Fuzzy_Matches", index=False)      # Matches only
    matched_details.to_excel(writer, sheet_name="Matched_Details", index=False)  # Full matched records

print(f"✅ Final Excel Report Saved: {output_path}")


🔍 Matching Groups: 100%|██████████| 88321/88321 [04:33<00:00, 322.56it/s]


✅ Final Excel Report Saved: /kaggle/working/Invoice_Fuzzy_Report_With_Raw.xlsx


In [5]:
# 📦 Required Libraries
import pandas as pd
from difflib import SequenceMatcher
from itertools import combinations
from tqdm import tqdm

# ✅ Load Excel XLSB File (Update path as per Kaggle)
file_path = "/kaggle/input/invoice-dump-24-25-xlsb/Invoice_Dump_24-25.xlsb"
raw_df = pd.read_excel(file_path, sheet_name=0, engine='pyxlsb')

# ✅ Extract only required columns for fuzzy matching
df = raw_df[['PAYEE_CODE', 'INVOICE_NO', 'TOTAL_AMOUNT']].dropna().copy()

# ✅ Convert to string for comparison
df['PAYEE_CODE'] = df['PAYEE_CODE'].astype(str)
df['INVOICE_NO'] = df['INVOICE_NO'].astype(str)

# ✅ Fuzzy Matching Logic – Group by PAYEE_CODE and TOTAL_AMOUNT
matches = []
grouped = df.groupby(['PAYEE_CODE', 'TOTAL_AMOUNT'])

for (payee, amount), group_df in tqdm(grouped, desc="🔍 Matching Groups"):
    inv_list = group_df['INVOICE_NO'].drop_duplicates().tolist()
    for inv1, inv2 in combinations(inv_list, 2):
        if inv1 != inv2:
            sim = SequenceMatcher(None, inv1, inv2).ratio()
            if sim >= 0.90:
                matches.append((payee, inv1, inv2, amount, round(sim, 2)))

# ✅ Convert match results to DataFrame
fuzzy_df = pd.DataFrame(matches, columns=['Payee_Code', 'Invoice_1', 'Invoice_2', 'Amount', 'Similarity'])

# ✅ Extract detailed matched records from full raw data
matched_invoice_ids = set(fuzzy_df['Invoice_1']).union(set(fuzzy_df['Invoice_2']))
matched_details = raw_df[raw_df['INVOICE_NO'].astype(str).isin(matched_invoice_ids)]

# ✅ Save to Excel in Kaggle Output Folder
output_path = "/kaggle/working/Invoice_Fuzzy_Report_Minimal.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    raw_df.to_excel(writer, sheet_name="Raw_Data", index=False)              # Full untouched raw data
    fuzzy_df.to_excel(writer, sheet_name="Fuzzy_Matches", index=False)      # Only matches
    matched_details.to_excel(writer, sheet_name="Matched_Details", index=False)  # Full matched records

print(f"✅ Final Excel saved to: {output_path}")


🔍 Matching Groups: 100%|██████████| 88321/88321 [02:57<00:00, 496.23it/s] 


✅ Final Excel saved to: /kaggle/working/Invoice_Fuzzy_Report_Minimal.xlsx


In [6]:
# 📦 Required Libraries
import pandas as pd
from difflib import SequenceMatcher
from itertools import combinations
from tqdm import tqdm

# ✅ Load Excel XLSB File (update path as needed)
file_path = "/kaggle/input/invoice-dump-24-25-xlsb/Invoice_Dump_24-25.xlsb"
raw_df = pd.read_excel(file_path, sheet_name=0, engine='pyxlsb')

# ✅ Work only with relevant columns for fuzzy logic
df = raw_df[['PAYEE_CODE', 'INVOICE_NO', 'TOTAL_AMOUNT']].dropna().copy()

# Convert types for fuzzy comparison
df['PAYEE_CODE'] = df['PAYEE_CODE'].astype(str)
df['INVOICE_NO'] = df['INVOICE_NO'].astype(str)

# ✅ Fuzzy Matching: Group by PAYEE_CODE and TOTAL_AMOUNT
matches = []
grouped = df.groupby(['PAYEE_CODE', 'TOTAL_AMOUNT'])

for (payee, amount), group_df in tqdm(grouped, desc="🔍 Matching Groups"):
    invoice_list = group_df['INVOICE_NO'].drop_duplicates().tolist()
    for inv1, inv2 in combinations(invoice_list, 2):
        if inv1 != inv2:
            sim = SequenceMatcher(None, inv1, inv2).ratio()
            if sim >= 0.90:
                matches.append((payee, inv1, inv2, amount, round(sim, 2)))

# ✅ Build Fuzzy Match DataFrame
fuzzy_df = pd.DataFrame(matches, columns=['Payee_Code', 'Invoice_1', 'Invoice_2', 'Amount', 'Similarity'])

# ✅ Get full raw rows for matched invoices (as-is from the original file)
matched_invoice_ids = set(fuzzy_df['Invoice_1']).union(set(fuzzy_df['Invoice_2']))
matched_full_rows = raw_df[raw_df['INVOICE_NO'].astype(str).isin(matched_invoice_ids)]

# ✅ Save all to Excel in /kaggle/working
output_path = "/kaggle/working/Invoice_Fuzzy_Report_Clean.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    raw_df.to_excel(writer, sheet_name="Raw_Data", index=False)               # Full original data
    fuzzy_df.to_excel(writer, sheet_name="Fuzzy_Matches", index=False)       # Fuzzy matches
    matched_full_rows.to_excel(writer, sheet_name="Matched_Details", index=False)  # Raw rows of matched invoices

print(f"✅ Final Excel saved to: {output_path}")


🔍 Matching Groups: 100%|██████████| 88321/88321 [03:08<00:00, 468.24it/s] 


✅ Final Excel saved to: /kaggle/working/Invoice_Fuzzy_Report_Clean.xlsx


In [7]:
# 📦 Required Libraries
import pandas as pd
from difflib import SequenceMatcher
from itertools import combinations
from tqdm import tqdm

# ✅ Load full Excel file with all columns intact
file_path = "/kaggle/input/invoice-dump-24-25-xlsb/Invoice_Dump_24-25.xlsb"
raw_df = pd.read_excel(file_path, sheet_name=0, engine='pyxlsb')  # This is the full raw data

# ✅ Extract just the three columns needed for matching logic
match_df = raw_df[['PAYEE_CODE', 'INVOICE_NO', 'TOTAL_AMOUNT']].dropna().copy()

# Convert to strings for fuzzy comparison
match_df['PAYEE_CODE'] = match_df['PAYEE_CODE'].astype(str)
match_df['INVOICE_NO'] = match_df['INVOICE_NO'].astype(str)

# ✅ Group by PAYEE_CODE and TOTAL_AMOUNT to do efficient fuzzy matching
matches = []
grouped = match_df.groupby(['PAYEE_CODE', 'TOTAL_AMOUNT'])

for (payee, amount), group in tqdm(grouped, desc="🔍 Processing Groups"):
    invoice_list = group['INVOICE_NO'].drop_duplicates().tolist()
    for inv1, inv2 in combinations(invoice_list, 2):
        if inv1 != inv2:
            sim = SequenceMatcher(None, inv1, inv2).ratio()
            if sim >= 0.90:
                matches.append((payee, inv1, inv2, amount, round(sim, 2)))

# ✅ Convert matches into a DataFrame
fuzzy_df = pd.DataFrame(matches, columns=['Payee_Code', 'Invoice_1', 'Invoice_2', 'Amount', 'Similarity'])

# ✅ Extract full rows from the raw data where invoices matched
matched_invoice_ids = set(fuzzy_df['Invoice_1']).union(set(fuzzy_df['Invoice_2']))
matched_details = raw_df[raw_df['INVOICE_NO'].astype(str).isin(matched_invoice_ids)]

# ✅ Save all 3 sheets to Excel in Kaggle's working directory
output_path = "/kaggle/working/Invoice_Fuzzy_Report_Complete.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    raw_df.to_excel(writer, sheet_name="Raw_Data", index=False)               # FULL Raw Data (All Columns)
    fuzzy_df.to_excel(writer, sheet_name="Fuzzy_Matches", index=False)       # Matches (Invoice_1, Invoice_2, etc.)
    matched_details.to_excel(writer, sheet_name="Matched_Details", index=False)  # Full Rows for Matched Invoices

print(f"✅ Final Excel saved with full raw + matches: {output_path}")


🔍 Processing Groups: 100%|██████████| 88321/88321 [03:00<00:00, 490.01it/s] 


✅ Final Excel saved with full raw + matches: /kaggle/working/Invoice_Fuzzy_Report_Complete.xlsx


In [8]:
# 📦 Required Libraries
import pandas as pd
from difflib import SequenceMatcher
from itertools import combinations
from tqdm import tqdm

# ✅ Load the full original XLSB data (not filtered)
file_path = "/kaggle/input/invoice-dump-24-25-xlsb/Invoice_Dump_24-25.xlsb"
raw_df = pd.read_excel(file_path, sheet_name=0, engine='pyxlsb')  # FULL DATA with all columns

# ✅ Create a working DataFrame with ONLY 3 columns for fuzzy logic (not for saving)
match_df = raw_df[['PAYEE_CODE', 'INVOICE_NO', 'TOTAL_AMOUNT']].dropna().copy()

# Convert types to string for comparison
match_df['PAYEE_CODE'] = match_df['PAYEE_CODE'].astype(str)
match_df['INVOICE_NO'] = match_df['INVOICE_NO'].astype(str)

# ✅ Fuzzy Matching grouped by PAYEE_CODE and TOTAL_AMOUNT
matches = []
grouped = match_df.groupby(['PAYEE_CODE', 'TOTAL_AMOUNT'])

for (payee, amount), group in tqdm(grouped, desc="🔍 Matching Groups"):
    invoice_list = group['INVOICE_NO'].drop_duplicates().tolist()
    for inv1, inv2 in combinations(invoice_list, 2):
        if inv1 != inv2:
            sim = SequenceMatcher(None, inv1, inv2).ratio()
            if sim >= 0.90:
                matches.append((payee, inv1, inv2, amount, round(sim, 2)))

# ✅ Build Match Result DataFrame
fuzzy_df = pd.DataFrame(matches, columns=['Payee_Code', 'Invoice_1', 'Invoice_2', 'Amount', 'Similarity'])

# ✅ Extract full matched rows from original raw_df (not filtered)
matched_invoice_ids = set(fuzzy_df['Invoice_1']).union(set(fuzzy_df['Invoice_2']))
matched_rows_full = raw_df[raw_df['INVOICE_NO'].astype(str).isin(matched_invoice_ids)]

# ✅ Save to Excel — Full Raw + Matches + Matched Details
output_path = "/kaggle/working/Invoice_Fuzzy_Report_Corrected.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    raw_df.to_excel(writer, sheet_name="Raw_Data", index=False)                  # FULL Raw Data (All Columns)
    fuzzy_df.to_excel(writer, sheet_name="Fuzzy_Matches", index=False)          # Fuzzy Match Results
    matched_rows_full.to_excel(writer, sheet_name="Matched_Details", index=False)  # Full raw rows of matched invoices

print(f"✅ Fixed and Full Excel report saved to: {output_path}")


🔍 Matching Groups: 100%|██████████| 88321/88321 [03:02<00:00, 484.05it/s] 


✅ Fixed and Full Excel report saved to: /kaggle/working/Invoice_Fuzzy_Report_Corrected.xlsx
